# 4. Citybenches: Implementation

In [8]:
#Necessary packages
import osmnx as ox
import networkx as nx
import geopandas 
import matplotlib.pyplot as plt
import pandas as pd

## Results of Data Wrangling Round 1

In [9]:
#Import data frame, adding 'New York City' to every field in Address-column
df = pd.read_excel('citybenchlocationsv03.xlsx')
df['Address'] = df['Address'] + ', New York City'

In [10]:
#Subset original data set
addressborough = df[['Address', 'Borough']]

In [11]:
#Further subsetting
address = df[['Address']]

## 4.1 Data Wrangling: Round 2

In [27]:
#Adding a column with Geocodes to df
addressgeo = []
for i in address['Address']:
    addressgeo.append(ox.geocode(i))
address.insert(2, 'Geocode', addressgeo, True)

Exception: Nominatim geocoder returned no results for query "41-51 Main Street (136-05 Sanford
Avenue), New York City"

In [ ]:
#Choosing a graph that contains all ten addresses (suboptimal solution)
G = ox.graph_from_point((40.74619633333333, -73.983977), network_type = 'walk', distance = 100, simplify = False)

In [25]:
#Adding a column with Nodes to df
addressnode = []
for i in addressgeo['Geocode']:
    addressnode.append(ox.get_nearest_node(G, i))    
address.insert(3, 'Node', addressnode, True)

TypeError: list indices must be integers or slices, not str